## Double The Fun!


link to [Double The Fun!]

In [ ]:
using JuMP, Gurobi

In [ ]:
# global constants
g_maxAngVel = 1 # move in azimuth or elevation at max rate of 1 rad / sec
g_maxScanRate = 1000 # scans points at the rate of 1000 pts/sec

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in PCS
# output: nx3 list of points in MCS
function pcs_to_mcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = inv(mcs) * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in MCS
# output: nx3 list of points in PCS
function mcs_to_pcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = mcs * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian to spherical
# input:  pts: nx3 list of xyz (cartesian) points
# output: nx3 list of spherical points (range, theta, phi)
function xyz_to_sph(pts)
    r = [vecnorm(pts[i,:]) for i=1:size(pts,1)]
    r2= [vecnorm(pts[i,1:2]) for i=1:size(pts,1)]
    t = atan2.(r2, pts[:,3])
    p = atan2.(pts[:,2], pts[:,1])
    return [r t p]
end;

# function to convert points from spherical to cartesian
# input:  pts: nx3 list of (r,t,p) (spherical) points
# output: nx3 list of cartesian points (xyz)
function sph_to_xyz(pts)
    # multiply by inverse of the LR's location/orientation
    x = pts[:,1] .* sin.(pts[:,2]) .* cos.(pts[:,3]) # rsinθcosϕ
    y = pts[:,1] .* sin.(pts[:,2]) .* sin.(pts[:,3]) # rsinθsinϕ
    z = pts[:,1] .* cos.(pts[:,2])                   # rcosθ
    return [x y z]
end;


# function to simulate a scan from one spherical coord to another
# input: from: spherical coordinate triple (radius, theta, phi) to start from
#        to:   spherical coordinate triple (radius, theta, phi) to end at
# output: n x 3 list of spherical points
function scan_from_to(from, to)
    delta = to .- from
    time = maximum(abs.(delta[2:3])) / g_maxAngVel
    qty = floor(time * g_maxScanRate)
    #@printf("Scanning from (%f,%f,%f) to (%f,%f,%f), %d points\n", from[1], from[2], from[3], to[1], to[2], to[3], qty)
    rgs = linspace.(from, to, maximum([qty,5])) # want at least 5 points.
    return [collect(rgs[1]) collect(rgs[2]) collect(rgs[3])]
end

# function to simulate a scan through a list of (spherical) points
# input:  list of points (spherical) in the order to be visited
# output: list of points (spherical) of the simulated scan points
function scan_tour(pts)
    ret = []
    for i = 1:size(pts,1)-1
        line = scan_from_to(pts[i,:], pts[i+1,:])
        ret = [ret; line]
    end
    return ret
end

# function that computes the travel time from spherical coordinate 'from' to 'to'
# (assumes instantaneous acceleration to max velocity)
travel_time(from, to) = maximum(abs.((to .- from)[2:3])) / g_maxAngVel

# function that takes a list of nx3 spherical coordinates and returns an n x n 
# matrix where A(i,j) where  the time it takes to move from point i to point j
cost_matrix(sph) = [travel_time(sph[i,:],sph[j,:]) for i=1:size(sph,1), j=1:size(sph,1)]


In [79]:
function TSP2_LazyIterSubtour(cost1, cost2)
    n = size(cost1,1) # number of features
    # to form a path, add a dummy node of cost 0 to everywhere.
    # add two dummy nodes because we have 2 laser radars
    for i = 1:2
        cost1 = [cost1 zeros(size(cost1,1),1)]
        cost1 = [cost1 ; zeros(1, size(cost1,2))]
        cost2 = [cost2 zeros(size(cost2,1),1)]
        cost2 = [cost2 ; zeros(1, size(cost2,2))]
        n += 1
    end

    m = Model(solver=GurobiSolver(OutputFlag=0))
    @variable(m, z1[1:n, 1:n], Bin) # LR1 path
    @variable(m, z2[1:n, 1:n], Bin) # LR2 path
    @variable(m, s[1:n], Bin) # selects LR1 or LR2 for each feature
    @variable(m, t >= 0) # for epigraph trick
    
    @objective(m, Min, t) # for epigraph trick to minimize maximum path cost
    @constraint(m, sum(s.*z1.*cost1) <= t) # cost of LR1 is less than t
    @constraint(m, sum((1-s).*z2.*cost2) <= t) # cost of lR2 is less than t
    
    # first LR's tour must not use the second dummy node (n)
    @constraint(m, [i in 1:n-1], sum(z1[i,:]) == s[i]) # leave each feature once
    @constraint(m, [j in 1:n-1], sum(z1[:,j]) == s[j]) # arrive at each feature once
    @constraint(m, sum(z1[n,:]) == 0) # force z1 to not take second dummy node
    @constraint(m, [ii in 1:n], z1[ii,ii] == 0) # can't transition to self

    # second LR's tour must not use the first dummy node (n-1)
    for i=1:n 
        if i != n-1
            @constraint(m, sum(z2[i,:]) == (1-s[i])) # leave each feature once
        end
    end
    for j = 1:n
        if j != n-1
            @constraint(m, sum(z2[:,j]) == (1-s[j])) # arrive at each feature once
        end
    end     
    @constraint(m, sum(z2[n-1,:]) == 0) # force z2 to not take first dummy node
    @constraint(m, [ii in 1:n], z2[ii,ii] == 0) # can't transition to self
    
    num_additional_constraints = 0
    function find_subtour(idxs,start)
        subtour = [start]
        while true
            next = idxs[subtour[end]]
            push!(subtour,next)
            if start == next
                break
            end
        end
        return subtour
    end

    function find_subtours2(idxs)
        remaining = filter(x->x!=0, idxs)
        subtours = []
        while length(remaining) > 0 
            subtour = find_subtour(idxs, remaining[1])
            push!(subtours, subtour)
            remaining = setdiff(remaining, subtour)
        end
        return subtours
    end
    
    function check_for_subtours(z, cb)
        soln = getvalue(z)
        # check for subtours
        idxs = []
        for i = 1:n
            lst = find(soln[i,:] .>= 0.5)
            push!(idxs, length(lst) > 0 ? lst[1] : 0)
        end
        subtours = find_subtours2(idxs)
        if length(subtours) == 1
            solved = true;
        else
            # add constraints to prohibit these subtours
            for subtour in subtours
                #println("lazy eliminate subtour: ", subtour)
                len = length(subtour)
                # prohibit these particular features from being a tour
                @lazyconstraint(cb, sum( z[subtour[i],subtour[i+1]] for i = 1:len-1 ) <= len-2)
                # and prohibit the reverse
                @lazyconstraint(cb, sum( z[subtour[i+1],subtour[i]] for i = 1:len-1 ) <= len-2)
                num_additional_constraints += 2
            end            
        end        
    end
        
    function subtour_callback(cb)
        check_for_subtours(z1,cb)
        check_for_subtours(z2,cb)
    end

    function remove_dummy_node(tour)
        len = length(tour)
        idx_dummy = indmax(tour)
        return [tour[(i+idx_dummy)%len+1] for i=0:len-2]
    end

    function get_tour(soln, cost, choice)
        idxs= []
        for i = 1:n
            lst = find(soln[i,:] .>= 0.5)
            push!(idxs, length(lst) > 0 ? lst[1] : 0)
        end
        tour = find_subtours(idxs)[1]
        tour = remove_dummy_node(tour[1:end-1])
        return [tour, sum(choice.*soln.*cost)]
    end
    
    addlazycallback(m, subtour_callback)
    solve(m)
    
    # extract feature order
    choice = getvalue(s)
    t1 = get_tour(getvalue(z1), cost1, choice)
    t2 = get_tour(getvalue(z2), cost2, (1.-choice))
    return [t1, t2]
end;

# test harness
function test2(featureFile, locationFile1, locationFile2)            
    features = readcsv(featureFile); # feature file
    pts_pcs = convert(Array{Float64, 2}, features[:,2:4]) # feature locations in PCS

    lr1_position = readcsv(locationFile1) # transformation matrix for LR's positionend
    lr2_position = readcsv(locationFile2) # transformation matrix for LR's positionend

    start = 1
    num = 18
    # spherical coordinates with origin at laser radar
    sph1 = xyz_to_sph(pcs_to_mcs_xyz(lr1_position, pts_pcs))
    cost1 = cost_matrix(sph1[start:start+num,:]) # cost to travel from feature i to feature j

    sph2 = xyz_to_sph(pcs_to_mcs_xyz(lr2_position, pts_pcs))
    cost2 = cost_matrix(sph2[start:start+num,:]) # cost to travel from feature i to feature j
    
    ret = TSP2_LazyIterSubtour(cost1, cost2)
    println("LR1 measures path: ", ret[1][1], " in ", ret[1][2], " seconds.")
    scan = mcs_to_pcs_xyz(lr1_position, sph_to_xyz(scan_tour(sph1[ret[1][1],:])))
    fname = @sprintf("MultiTSP_LR%d_Scan_%d-%d.csv", 1, start, start+num)
    writecsv(fname, scan)

    println("LR2 measures path: ", ret[2][1], " in ", ret[2][2], " seconds.")
    scan = mcs_to_pcs_xyz(lr2_position, sph_to_xyz(scan_tour(sph2[ret[2][1],:])))
    fname = @sprintf("MultiTSP_LR%d_Scan_%d-%d.csv", 2, start, start+num)
    writecsv(fname, scan)
end
        
@time(test2("50_Holes_FrontLeft.csv", "LocationFront.csv", "LocationLeft.csv"))

Academic license - for non-commercial use only
LR1 measures path: [1, 10, 15, 13, 4, 7, 17, 8, 9, 6, 2, 16, 11] in 0.6354611540272634 seconds.
LR2 measures path: [14, 12, 18, 3, 5, 19] in 0.6239931571551316 seconds.
155.177331 seconds (585.58 k allocations: 51.152 MiB)


link to [Double The Fun!](#Double-The-Fun!)